# Model for correcting misspellea words

This part of the notebook was taken from https://github.com/deepmipt/raai_summer_school_nlp_2021. In this part, you can find the demonstration of how to work with transformers

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.9MB/s 
     |████████████████████████████████| 901kB 55.8MB/s 
     |████████████████████████████████| 3.3MB 56.1MB/s 


В современной компьютерной лингвистике вычисление вероятности текста производится в основном за счёт нейронных, а не энграммных моделей. Существует много разновидностей архитектур, мы рассмотрим 2: левостороннюю модель  `gpt2` (её облегчённую версию `distilgpt2`) и языковую модель с пропусками `BERT`.

## Односторонние языковые модели

Вначале создадим токенизатор и применим его к данным

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
text = "The Starship prototype descended under active aerodynamic control, accomplished by four vehicles."
tokenization = tokenizer(text)
print(tokenization)

{'input_ids': [464, 40172, 14879, 23667, 739, 4075, 9551, 34743, 1630, 11, 13013, 416, 1440, 5672, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
sents = [
    "Yesterday, all my troubles seemed so far away.",
    "I only want to say, if there is a way, take away this cup of poison, 'cause it burns me.",
    "We do not need your education, we do not need your thought control.",
    "When the light begins to change, I sometimes feel a little strange, a little anxious when it's dark."         
]
for elem in tokenizer(sents)["input_ids"]:
    print(elem)

[28065, 11, 477, 616, 14979, 3947, 523, 1290, 1497, 13]
[40, 691, 765, 284, 910, 11, 611, 612, 318, 257, 835, 11, 1011, 1497, 428, 6508, 286, 8764, 11, 705, 25587, 340, 20246, 502, 13]
[1135, 466, 407, 761, 534, 3707, 11, 356, 466, 407, 761, 534, 1807, 1630, 13]
[2215, 262, 1657, 6140, 284, 1487, 11, 314, 3360, 1254, 257, 1310, 6283, 11, 257, 1310, 18116, 618, 340, 338, 3223, 13]


In [7]:
tokenizer.pad_token = tokenizer.eos_token
for elem in tokenizer(sents, return_tensors="pt", padding=True)["input_ids"]:
    print(elem)

tensor([28065,    11,   477,   616, 14979,  3947,   523,  1290,  1497,    13,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256])
tensor([   40,   691,   765,   284,   910,    11,   611,   612,   318,   257,
          835,    11,  1011,  1497,   428,  6508,   286,  8764,    11,   705,
        25587,   340, 20246,   502,    13])
tensor([ 1135,   466,   407,   761,   534,  3707,    11,   356,   466,   407,
          761,   534,  1807,  1630,    13, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256])
tensor([ 2215,   262,  1657,  6140,   284,  1487,    11,   314,  3360,  1254,
          257,  1310,  6283,    11,   257,  1310, 18116,   618,   340,   338,
         3223,    13, 50256, 50256, 50256])


 Загрузим модель на видеокарту.

In [8]:
from transformers import AutoModelWithLMHead, AutoModel

model = AutoModelWithLMHead.from_pretrained("gpt2").to("cuda")
type(model)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:847: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [11]:
top_log_probs, top_indexes = torch.topk(logits[0], k=5, dim=-1)
top_indexes = top_indexes.cpu().numpy()
for i in range(len(logits[0])-1):
    print("<BEGIN> " + " ".join([x.strip("ĠĊ") for x in tokens[:i]]))
    curr_top_tokens = tokenizer.convert_ids_to_tokens(top_indexes[i])
    for index, token in zip(top_indexes[i], curr_top_tokens):
        token = token.strip("ĠĊ")
        print(f"{token}:{probs[i,index]:.3f}", end=" ")
    print("")

<BEGIN> 
:0.062 The:0.038 ":0.024 A:0.019 I:0.018 
<BEGIN> The
first:0.010 U:0.009 following:0.008 United:0.006 US:0.005 
<BEGIN> The Starship
Enterprise:0.213 Tro:0.212 Trooper:0.021 Ant:0.008 Crew:0.008 
<BEGIN> The Starship prototype
is:0.146 was:0.063 of:0.052 has:0.044 ,:0.042 
<BEGIN> The Starship prototype descended
into:0.391 to:0.135 from:0.115 on:0.041 in:0.019 
<BEGIN> The Starship prototype descended under
the:0.259 a:0.132 heavy:0.109 its:0.050 fire:0.041 
<BEGIN> The Starship prototype descended under active
investigation:0.103 fire:0.084 development:0.066 radar:0.043 construction:0.038 
<BEGIN> The Starship prototype descended under active aer
odynamic:0.790 odynamics:0.127 ob:0.036 on:0.022 opl:0.007 
<BEGIN> The Starship prototype descended under active aer odynamic
drag:0.327 control:0.104 braking:0.033 and:0.027 stress:0.025 
<BEGIN> The Starship prototype descended under active aer odynamic control
,:0.127 and:0.073 during:0.066 on:0.064 at:0.062 
<BEGIN> The Starsh

Теперь посмотрим, насколько модель знает грамматику.

In [12]:
 texts = [
     "Alexandra is very proud of herself.", "Alexandra is very proud of himself.",
     "Alexander is very proud of herself.", "Alexander is very proud of himself.",
     "Alexandra is very proud of she.", "Alexandra is very proud of her.",
     "Alexandra is very proud of her son."
 ]
 tokenizer.pad_token = tokenizer.eos_token
 batch = tokenizer(texts, return_tensors="pt", padding=True).to("cuda")
 # добавляем индекс начала строки (склейка массивов по первой координате)
 batch["input_ids"] = torch.cat([
    torch.ones_like(batch["input_ids"][:,:1])*tokenizer.bos_token_id, 
    batch["input_ids"]
 ], dim=1)
#  batch["attention_mask"] = torch.cat([
#     torch.ones_like(batch["attention_mask"][:,:1]),
#     batch["attention_mask"]
#  ], dim=-1)
 with torch.no_grad():
    logits = model(batch["input_ids"])["logits"]
 probs = torch.softmax(logits, dim=-1).cpu().numpy()
 print(probs.shape)

(7, 10, 50257)


In [13]:
for i, text in enumerate(texts):
    print(text)
    text_token_ids = batch["input_ids"][i,1:]
    text_tokens = [x.strip("ĠĊ") for x in tokenizer.convert_ids_to_tokens(text_token_ids)]
    for j, (index, token) in enumerate(zip(text_token_ids, text_tokens)):
        print(f"{token}:{probs[i,j,index]:.3f}", end=" ")
    print("")

Alexandra is very proud of herself.
Alex:0.000 andra:0.050 is:0.014 very:0.004 proud:0.028 of:0.579 herself:0.040 .:0.215 <|endoftext|>:0.002 
Alexandra is very proud of himself.
Alex:0.000 andra:0.050 is:0.014 very:0.004 proud:0.028 of:0.579 himself:0.002 .:0.213 <|endoftext|>:0.002 
Alexander is very proud of herself.
Alexander:0.000 is:0.011 very:0.004 proud:0.030 of:0.679 herself:0.001 .:0.205 <|endoftext|>:0.002 <|endoftext|>:0.000 
Alexander is very proud of himself.
Alexander:0.000 is:0.011 very:0.004 proud:0.030 of:0.679 himself:0.020 .:0.199 <|endoftext|>:0.002 <|endoftext|>:0.000 
Alexandra is very proud of she.
Alex:0.000 andra:0.050 is:0.014 very:0.004 proud:0.028 of:0.579 she:0.001 .:0.002 <|endoftext|>:0.008 
Alexandra is very proud of her.
Alex:0.000 andra:0.050 is:0.014 very:0.004 proud:0.028 of:0.579 her:0.356 .:0.005 <|endoftext|>:0.003 
Alexandra is very proud of her son.
Alex:0.000 andra:0.050 is:0.014 very:0.004 proud:0.028 of:0.579 her:0.356 son:0.009 .:0.165 


## Writing model for correcting misspelled words

Let`s load library with English dictionary


In [14]:
!apt install -qq enchant
!pip install pyenchant

The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspe

Here we write the function for finding misspelled words. If you want to correct not only misprint but also grammatical errors you should comment string (`if not d.check(word_list[i]):`). Unfortunately if you use the free version of colab commenting on the string lead to 'RUNTIME ERROR'

In [41]:
import enchant
#function for finding misspeled word in sentence
def find_misspeled(sentance):
  word_list = sentance.split(' ')
  misspelled_words = dict()
  #print(len(word_list))
  for i in range(len(word_list)):
    if not d.check(word_list[i]):
      misspelled_words[word_list[i]] = i
  return misspelled_words

In [16]:
#check fuction
### check the useful function in package enchant
#load english dictionary
d = enchant.Dict("en_US")
#find misspelled word
find_misspeled('He is intelligen')

{'intelligen': 2}

In [34]:
#generate possible words
d.suggest('intelligen')

['intelligent', 'intelligence', 'intelligible', 'intelligibly', 'belligerent']

Here we write a function for generating sentences with possible correct words

In [35]:
def generate_correction(sentance):
  sentance_list = [sentance]
  possible_correction_list = []
  word_dict = find_misspeled(sentance)
  
  for word in word_dict.keys():
    for sent in sentance_list:
      word_list = sent.split(' ')
      for correction_word in d.suggest(word):
        new_sentance = ' '.join(word_list[:word_dict[word]] + [correction_word] + word_list[word_dict[word]+1:])
        possible_correction_list  += [new_sentance]
    #print(sentance_list, word)
    sentance_list = possible_correction_list
    possible_correction_list = []
    #print(sentance_list)
  return sentance_list

Let`s look at the examples

In [18]:
 #check function
 generate_correction('He is intelligen')

['He is intelligent',
 'He is intelligence',
 'He is intelligible',
 'He is intelligibly',
 'He is belligerent']

In [19]:
#check function
generate_correction('He liks intelligen peaple')

['He leeks intelligent people',
 'He leeks intelligent Peale',
 'He leeks intelligent leaper',
 'He leeks intelligent apple',
 'He leeks intelligent appeal',
 'He leeks intelligence people',
 'He leeks intelligence Peale',
 'He leeks intelligence leaper',
 'He leeks intelligence apple',
 'He leeks intelligence appeal',
 'He leeks intelligible people',
 'He leeks intelligible Peale',
 'He leeks intelligible leaper',
 'He leeks intelligible apple',
 'He leeks intelligible appeal',
 'He leeks intelligibly people',
 'He leeks intelligibly Peale',
 'He leeks intelligibly leaper',
 'He leeks intelligibly apple',
 'He leeks intelligibly appeal',
 'He leeks belligerent people',
 'He leeks belligerent Peale',
 'He leeks belligerent leaper',
 'He leeks belligerent apple',
 'He leeks belligerent appeal',
 'He ilks intelligent people',
 'He ilks intelligent Peale',
 'He ilks intelligent leaper',
 'He ilks intelligent apple',
 'He ilks intelligent appeal',
 'He ilks intelligence people',
 'He ilks 

In [20]:
 texts = generate_correction('He liks intelligen peaple')
 tokenizer.pad_token = tokenizer.eos_token
 batch = tokenizer(texts, return_tensors="pt", padding=True).to("cuda")
 # добавляем индекс начала строки (склейка массивов по первой координате)
 batch["input_ids"] = torch.cat([
    torch.ones_like(batch["input_ids"][:,:1])*tokenizer.bos_token_id, 
    batch["input_ids"]
 ], dim=1)
#  batch["attention_mask"] = torch.cat([
#     torch.ones_like(batch["attention_mask"][:,:1]),
#     batch["attention_mask"]
#  ], dim=-1)
 with torch.no_grad():
    logits = model(batch["input_ids"])["logits"]
 probs = torch.softmax(logits, dim=-1).cpu().numpy()
 print(probs.shape)

(250, 9, 50257)


Here we write code that for each subtokens in a sentence shows its logarithm of possibility to appear in the sentence

In [22]:

for i, text in enumerate(texts):
    print(text)
    text_token_ids = batch["input_ids"][i,1:]
    text_tokens = [x.strip("ĠĊ") for x in tokenizer.convert_ids_to_tokens(text_token_ids)]
    for j, (index, token) in enumerate(zip(text_token_ids, text_tokens)):
        if token != '<|endoftext|>':
          print(f"{token}:{math.log10(probs[i,j,index]):.3f}", end=" ")
    print("")

He leeks intelligent people
He:-2.584 le:-4.359 eks:-3.924 intelligent:-5.550 people:-2.089 
He leeks intelligent Peale
He:-2.584 le:-4.359 eks:-3.924 intelligent:-5.550 Pe:-5.157 ale:-3.447 
He leeks intelligent leaper
He:-2.584 le:-4.359 eks:-3.924 intelligent:-5.550 le:-3.197 aper:-4.373 
He leeks intelligent apple
He:-2.584 le:-4.359 eks:-3.924 intelligent:-5.550 apple:-4.334 
He leeks intelligent appeal
He:-2.584 le:-4.359 eks:-3.924 intelligent:-5.550 appeal:-4.481 
He leeks intelligence people
He:-2.584 le:-4.359 eks:-3.924 intelligence:-4.806 people:-3.943 
He leeks intelligence Peale
He:-2.584 le:-4.359 eks:-3.924 intelligence:-4.806 Pe:-5.932 ale:-3.097 
He leeks intelligence leaper
He:-2.584 le:-4.359 eks:-3.924 intelligence:-4.806 le:-3.803 aper:-4.011 
He leeks intelligence apple
He:-2.584 le:-4.359 eks:-3.924 intelligence:-4.806 apple:-5.632 
He leeks intelligence appeal
He:-2.584 le:-4.359 eks:-3.924 intelligence:-4.806 appeal:-5.404 
He leeks intelligible people
He:-2.5

So to find our correct sentance we should sum all logarithm of the possibility of each subtokens unless '<|endoftext|>' and find sentences with the max possibility

In [39]:
def generate_possible_sentence(sentance, best_three = False):
  texts = generate_correction(sentance)
  tokenizer.pad_token = tokenizer.eos_token
  batch = tokenizer(texts, return_tensors="pt", padding=True).to("cuda")
  # добавляем индекс начала строки (склейка массивов по первой координате)
  batch["input_ids"] = torch.cat([
      torch.ones_like(batch["input_ids"][:,:1])*tokenizer.bos_token_id, 
      batch["input_ids"]
  ], dim=1)
  #  batch["attention_mask"] = torch.cat([
  #     torch.ones_like(batch["attention_mask"][:,:1]),
  #     batch["attention_mask"]
  #  ], dim=-1)
  with torch.no_grad():
      logits = model(batch["input_ids"])["logits"]
  probs = torch.softmax(logits, dim=-1).cpu().numpy()
  #print(probs.shape)
  sentence_probability = dict()
  for i, text in enumerate(texts):
      #print(text)
      s = 1
      
      text_token_ids = batch["input_ids"][i,1:]
      text_tokens = [x.strip("ĠĊ") for x in tokenizer.convert_ids_to_tokens(text_token_ids)]
      for j, (index, token) in enumerate(zip(text_token_ids, text_tokens)):
          if token != '<|endoftext|>':
            #print(probs[i,j,index], j)
            s *= probs[i,j,index]
      sentence_probability[text] = s
      #print(s)
      #print("")
  markdict = sentence_probability
  marklist = sorted(markdict.items(), key=lambda x:x[1], reverse=True)
  sortdict = dict(marklist)
  a = list(sortdict.keys())
  print(a[0], math.log10(sortdict[a[0]]))
  if best_three:
    if len(a) > 1:
      print(a[1], math.log10(sortdict[a[1]]))
    if len(a) > 2:
      print(a[2], math.log10(sortdict[a[2]]))

let's look on example

In [42]:
generate_possible_sentence('He liks intelligen peapel')

He likes intelligent appeal -14.8900343748022


In [88]:
generate_possible_sentence('He likes intelligent peapel. They are really smart')

He likes intelligent appeal They are really smart -24.579315159329713


In [89]:
generate_possible_sentence('He was very hard person. H makes people angry')

He was very hard person. H makes people angry -27.266935062574156


In [90]:
generate_possible_sentence('He was very hard person. Hi makes people angry')

He was very hard person. Hi makes people angry -29.369194669865823


the number means the logarithm of the probability of the given right to offer

let's look on example of best three sentances

In [43]:
generate_possible_sentence('He liks intelligen poeple', best_three = True)

He likes intelligent people -10.859718182047587
He like intelligent people -12.039852992837014
He likes intelligence people -12.702763681423061


In [45]:
generate_possible_sentence('Nice to mete you')

Nice to mete you -14.887436033400595


In [49]:
generate_possible_sentence('Nice to ment you', best_three = True)

Nice to meet you -6.23056891547142
Nice to met you -9.22511285645124
Nice to sent you -11.983995502819349


In [ ]:
generate_possible_sentence('Where are you from')

Where are you from -5.822051746623529


In [50]:
generate_possible_sentence('Where are you frm', best_three = True)

Where are you from -5.822049168133126
Where are you fro -9.984711659688703
Where are you fr -10.156299643015004


In [52]:
generate_possible_sentence('Wher are you from')

Where are you from -5.822059694891647


In [53]:
generate_possible_sentence('Where are yu from')

Where are you from -5.822052646721279


In [54]:
generate_possible_sentence('Wher is you from')

Where is you from -8.00890321694338


In [55]:
generate_possible_sentence('Wher ar you from')

Where are you from -5.822054530763859


In [70]:
generate_possible_sentence('Excuce me, sar, you dropped your wallet')

Excuse me Sara you dropped your wallet -22.20794236369435


In [57]:
generate_possible_sentence('I dod not invit you to the pary.')

I did not invite you to the party -11.473779448857929


In [58]:
generate_possible_sentence('I m really sory')

I m really sorry -10.48651595341332


In [59]:
generate_possible_sentence('Im really sory')

I'm really sorry -5.891609285081461


In [ ]:
generate_possible_sentence('Wht do youthink?')

What do outhitting -13.863678994758327
Why do outhitting -14.010794880391714
Who do outhitting -15.131250615720894


In [ ]:
generate_possible_sentence('Oh, never mins')

In [64]:
generate_possible_sentence('Oh, never mins')

Oh never mind -7.874431643460552


If you have this error it means that there is no enough  memory on colab and you should restart everything:(

In [80]:
generate_possible_sentence('Im Masha')

RuntimeError: ignored

In [77]:
generate_possible_sentence('Im Jack')

In Jack -6.849314044971794


In [76]:
generate_possible_sentence('Im Jack. I have a son')

I'm Jack. I have a son -13.190372789967247


In [74]:
generate_possible_sentence('Im Pol. I have a son')

I'm Pol. I have a son -16.118025857377518


In [72]:
generate_possible_sentence('Im seven years old')

I'm seven years old -7.590789444221681


In [65]:
generate_possible_sentence('Thaks so muc for the birthdays money.')

Thanks so much for the birthdays money. -17.193170857781922


In [67]:
generate_possible_sentence('I really appreiate yor help.')

I really appreciate your help. -8.536285601209237


GPT2 does not know my name

In [82]:
generate_possible_sentence('I am Masha')

I am Tasha -8.588813247887964


In [83]:
generate_possible_sentence('I am Maria')

I am Maria -7.988472099562584


In [85]:
generate_possible_sentence('Input arrays to be multiplid.')


Input arrays to be multiplied -15.16632472214808
